# 5-3: DataFrame Manipulation

Now that we have the basics of `DataFrame`s sorted, we're ready to use Pandas manipulate our data at scale. This is where we use grouping, aggregation

We'll be using a log of DNS queries from [Zeek](https://zeek.org/), courtesy of the [Security Datasets](https://github.com/OTRF/Security-Datasets) repo.

DNS logs are a great example of when we want to use Pandas to analyze aggregated data. These logs will almost always be far too large to review manually.

Our `dns.log` is in fact a JSON file. You might think that means we need to import the `json` module.

But naaaaah, Pandas has a `.read_json()` method. Let's load this `DataFrame` up.

In [14]:
# Import as always
import pandas as pd

# Create our DataFrame
df = pd.read_json("dns.log")

## Exploratory Data Analysis

[**EDA**](https://towardsdatascience.com/exploratory-data-analysis-8fc1cb20fd15) is always our first step with a proper dataset. This process gives us a general sense for the size and scope of the dataset, as well as some basic statistical information.

To start, I like to use the `.shape` property, which tells us the number of rows and columns as a tuple.

In [15]:
# (rows, cols)
df.shape

(171, 29)

Okay, so not _that_ big, especially for DNS logs. Let's check the columns we have. We can do that with either `df.columns` or `df.info()`. I prefer the latter because it tells us the data type of each column.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   @stream      171 non-null    object 
 1   @system      171 non-null    object 
 2   @proc        171 non-null    object 
 3   ts           171 non-null    float64
 4   uid          171 non-null    object 
 5   id_orig_h    171 non-null    object 
 6   id_orig_p    171 non-null    int64  
 7   id_resp_h    171 non-null    object 
 8   id_resp_p    171 non-null    int64  
 9   proto        171 non-null    object 
 10  trans_id     171 non-null    int64  
 11  rtt          98 non-null     float64
 12  query        171 non-null    object 
 13  qclass       171 non-null    int64  
 14  qclass_name  171 non-null    object 
 15  qtype        171 non-null    int64  
 16  qtype_name   171 non-null    object 
 17  rcode        132 non-null    float64
 18  rcode_name   132 non-null    object 
 19  AA      

Ah, Zeek logs. So clean. So well-named.

These can take a while to get used to. At this point, it's a good idea to check a few rows to see what these look like. `df.head()` will print the first 5 events.

In [17]:
df.head()

,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,trans_id,rtt,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,auth,addl
0,dns,test-nsm,zeek,1.588317e+09,CyLse01Jig9yHofDXc,10.0.1.5,55979,10.0.0.4,53,udp,40230,0.064170,ctldl.windowsupdate.com,1,C_INTERNET,1,A,0.0,NOERROR,False,False,True,True,0,"[audownload.windowsupdate.nsatc.net, au.downlo...","[2725.0, 599.0, 899.0, 19.0, 19.0]",False,NaN,NaN
1,dns,test-nsm,zeek,1.588317e+09,CyLse01Jig9yHofDXc,10.0.1.5,55979,10.0.0.4,53,udp,40230,NaN,ctldl.windowsupdate.com,1,C_INTERNET,1,A,NaN,NaN,False,False,True,False,0,NaN,NaN,False,NaN,NaN
2,dns,test-nsm,zeek,1.588317e+09,C55VWq1ibAOPBspBn4,10.0.1.5,56259,10.0.0.4,53,udp,63265,0.015746,arc.msn.com,1,C_INTERNET,1,A,0.0,NOERROR,False,False,True,True,0,"[arc.msn.com.nsatc.net, 52.138.119.21]","[1072.0, 300.0]",False,NaN,NaN
3,dns,test-nsm,zeek,1.588317e+09,CD0rgw3qZTWQNLB1ik,10.0.1.5,64294,10.0.0.4,53,udp,1349,0.137844,img-prod-cms-rt-microsoft-com.akamaized.net,1,C_INTERNET,1,A,0.0,NOERROR,False,False,True,True,0,"[a1449.dscg2.akamai.net, 23.48.39.43, 23.48.39...","[299.0, 19.0, 19.0]",False,NaN,NaN
4,dns,test-nsm,zeek,1.588317e+09,CD0rgw3qZTWQNLB1ik,10.0.1.5,64294,10.0.0.4,53,udp,1349,NaN,img-prod-cms-rt-microsoft-com.akamaized.net,1,C_INTERNET,1,A,NaN,NaN,False,False,True,False,0,NaN,NaN,False,NaN,NaN


You may notice that between `proto` and `AA` columns is an ellipses. By default, Pandas will abbreviate output to make things easier to read. This is true for both rows and columns, but sometimes that's not what we want. In this case, I really _do_ want to see all 29 columns! I'm willing to scroll to the right!

To change this, we can use [`pd.set_option()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.set_option.html) to change `display.max_columns` to a value of our choosing. Let's do that and re-run `.head()`

In [18]:
# Increase max cols and re-run .head()
pd.set_option("display.max_columns", 30)
df.head()

,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,trans_id,rtt,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,auth,addl
0,dns,test-nsm,zeek,1.588317e+09,CyLse01Jig9yHofDXc,10.0.1.5,55979,10.0.0.4,53,udp,40230,0.064170,ctldl.windowsupdate.com,1,C_INTERNET,1,A,0.0,NOERROR,False,False,True,True,0,"[audownload.windowsupdate.nsatc.net, au.downlo...","[2725.0, 599.0, 899.0, 19.0, 19.0]",False,NaN,NaN
1,dns,test-nsm,zeek,1.588317e+09,CyLse01Jig9yHofDXc,10.0.1.5,55979,10.0.0.4,53,udp,40230,NaN,ctldl.windowsupdate.com,1,C_INTERNET,1,A,NaN,NaN,False,False,True,False,0,NaN,NaN,False,NaN,NaN
2,dns,test-nsm,zeek,1.588317e+09,C55VWq1ibAOPBspBn4,10.0.1.5,56259,10.0.0.4,53,udp,63265,0.015746,arc.msn.com,1,C_INTERNET,1,A,0.0,NOERROR,False,False,True,True,0,"[arc.msn.com.nsatc.net, 52.138.119.21]","[1072.0, 300.0]",False,NaN,NaN
3,dns,test-nsm,zeek,1.588317e+09,CD0rgw3qZTWQNLB1ik,10.0.1.5,64294,10.0.0.4,53,udp,1349,0.137844,img-prod-cms-rt-microsoft-com.akamaized.net,1,C_INTERNET,1,A,0.0,NOERROR,False,False,True,True,0,"[a1449.dscg2.akamai.net, 23.48.39.43, 23.48.39...","[299.0, 19.0, 19.0]",False,NaN,NaN
4,dns,test-nsm,zeek,1.588317e+09,CD0rgw3qZTWQNLB1ik,10.0.1.5,64294,10.0.0.4,53,udp,1349,NaN,img-prod-cms-rt-microsoft-com.akamaized.net,1,C_INTERNET,1,A,NaN,NaN,False,False,True,False,0,NaN,NaN,False,NaN,NaN


Behold! A scrollbar! Now we can see all the columns and determine where the data of interest lives.

Looks like for this dataset, `id_origin_p`, `query`, and `answers` have the most interesting data. That'd be the requesting IP address, the DNS query, and the responses, if any.

Why don't we make our column names a little nicer? We can rename columns with the `DataFrame`'s `.rename()` method. It takes a `dict` of shape `{"current_name": "new_name"}` for every column you want to change. This is passed as the `columns` arg. And don't forget, to make it stick in our current `DataFrame`, we need to use `inplace=True`.

I like to make sure my column names are clean right at the start of any Pandas work, and I often document the column names up front as well.

In [21]:
# Rename columns
df.rename(columns={"id_orig_h": "source_ip", "id_resp_h": "response_ip"}, inplace=True)
# Show results
df[["source_ip","response_ip"]].head()

,source_ip,response_ip
0,10.0.1.5,10.0.0.4
1,10.0.1.5,10.0.0.4
2,10.0.1.5,10.0.0.4
3,10.0.1.5,10.0.0.4
4,10.0.1.5,10.0.0.4


### Grouping and Aggregation

Ever made a Pivot Table in Excel? Not super fun, right? Turns out Pandas has similar capabilities with just a few method invocations. Very often we will want to group our data by a field. For example, what if we wanted to see _how many_ requests each IP in our dataset generated?

Welcome to `.groupby()`.

`.groupby()` takes a field or a list of fields to group by. However, the result is a little odd because it is [`groupby` object](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) that doesn't show tabular data yet. That's because we need to chain it with an **aggregation function**. There are many built-in functions like `.count()`, `.mean()`, and `.sum()`, just to name a few. Keep in mind of course that these are all **quantitative** functions. They have to be, because we're talking about combining multiple rows of data into a single row. The only way a computer could sensibly do so is with mathematical functions. 

Let's group by our source IP and look at the count of how many queries each source provided.

In [24]:
# Group by source
df.groupby("source_ip").count()

,@stream,@system,@proc,ts,uid,id_orig_p,response_ip,id_resp_p,proto,trans_id,rtt,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,auth,addl
source_ip,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10.0.0.5,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,8,3,3,8,5,1
10.0.1.4,36,36,36,36,36,36,36,36,36,36,18,36,36,36,36,36,28,28,36,36,36,36,36,18,18,36,10,2
10.0.1.5,118,118,118,118,118,118,118,118,118,118,73,118,118,118,118,118,87,87,118,118,118,118,118,73,73,118,14,13
10.0.1.6,9,9,9,9,9,9,9,9,9,9,4,9,9,9,9,9,9,9,9,9,9,9,9,4,4,9,5,1


Now that's a lot of noise, since Pandas ran a count on every. single. field.

If we want to clean it up, we can specify a column at the end. But while we're at it, I also like to chain `.count()` with `.sort_values()` to get a descending count. `.sort_values()` takes a `by` argument that tells it what field to sort by, and an optional `ascending` argument to toggle ascending/descending. When sorting, you want to make sure you use the field that has the count you actually want. Look closely at the table above—not every column for a given row has the same value!

In [27]:
# Group by source, but cleaner and descending
df.groupby("source_ip").count().sort_values(by="query", ascending=False)["query"]

source_ip
10.0.1.5    118
10.0.1.4     36
10.0.1.6      9
10.0.0.5      8
Name: query, dtype: int64

Very clean! But `groupby()` can also take a list. Let's try grouping by source IP _and_ query, then count it up. We'll use `uid` as our sort-by. We'll also just display that field for cleanliness. To make it render as a proper HTML table rather than plain text, we'll "trick" Pandas into thinking it has a list of columns by giving it a singleton list.

Oh also, since we kinda know this is going to be large, I'm going to pregame by expanding the `display.max_rows` setting.

In [33]:
# Increase max display rows
pd.set_option("display.max_rows", 100)

# Group by source AND query, count 'em, then sort descending
df.groupby(["source_ip", "query"]).count().sort_values(by="uid", ascending=False)[["uid"]]

uid
source_ip query                                                  
10.0.1.5  wpad.dmevals.local                                    7
10.0.1.4  wpad.dmevals.local                                    6
          v10.events.data.microsoft.com                         6
10.0.1.5  login.live.com                                        5
          _ldap._tcp.default-first-site-name._sites.dc._m...    4
          crl.verisign.com                                      4
10.0.1.4  wec.dmevals.local                                     4
          wdcp.microsoft.com                                    4
          settings-win.data.microsoft.com                       4
10.0.1.5  prisoner.iana.org                                     4
          sv.symcd.com                                          4
          ecs.office.com                                        4
          nexus.passport.com                                    4
          nexusrules.officeapps.live.com                        4
          ocsp.verisign.com                                     4
10.0.1.4  ctldl.windowsupdate.com                               4
10.0.1.5  self.events.data.microsoft.com                        3
          login.microsoftonline.com                             3
          ctldl.windowsupdate.com                               3
          arc.msn.com                                           3
10.0.1.6  wpad.dmevals.local                                    3
10.0.0.5  wpad.dmevals.local                                    3
10.0.1.5  _ldap._tcp.default-first-site-name._sites.dc._m...    2
          mobile.pipe.aria.microsoft.com                        2
10.0.1.4  _ldap._tcp.dc._msdcs.scranton.dmevals.local           2
10.0.1.5  s2.symcb.com                                          2
          img-prod-cms-rt-microsoft-com.akamaized.net           2
          mrodevicemgr.officeapps.live.com                      2
10.0.1.4  _ldap._tcp.default-first-site-name._sites.dc._m...    2
          _ldap._tcp.pdc._msdcs.dmevals.local                   2
10.0.1.5  wsrksv4zpzzqe.blob.core.windows.net                   2
10.0.1.4  arc.msn.com                                           2
10.0.1.5  roaming.officeapps.live.com                           2
          d.docs.live.net                                       2
          s.symcd.com                                           2
          ts-ocsp.ws.symantec.com                               2
10.0.1.6  wec.dmevals.local                                     2
10.0.1.5  v10.events.data.microsoft.com                         2
          array615.prod.do.dsp.mp.microsoft.com                 2
          officecdn.microsoft.com                               2
          officeclient.microsoft.com                            2
          _ldap._tcp.pdc._msdcs.dmevals.local                   2
          watson.telemetry.microsoft.com                        2
          wec.dmevals.local                                     2
          _ldap._tcp.dc._msdcs.utica.dmevals.local              2
10.0.1.6  newyork.dmevals.local                                 1
          _ldap._tcp.pdc._msdcs.dmevals.local                   1
10.0.1.5  store-images.s-microsoft.com                          1
          scranton.dmevals.local                                1
          teams.microsoft.com                                   1
10.0.1.6  _ldap._tcp.default-first-site-name._sites.dc._m...    1
          _ldap._tcp.dc._msdcs.nashua.dmevals.local             1
10.0.1.5  utica.dmevals.local                                   1
          rdfepirv2bl3prdstr02.blob.core.windows.net            1
          www.bing.com                                          1
          v20.events.data.microsoft.com                         1
10.0.0.5  _ldap._tcp.dc._msdcs.warehouse.dmevals.local          1
10.0.1.5  newyork.dmevals.local                                 1
          _ldap._tcp.default-first-site-name._sites.gc._m...    1
10.0.0.5  _ldap._tcp.pdc._msdcs.dmev

## Filtering Values

That's a lot, right? And likely it'd be more than we need, especially when reviewing DNS queries. We can filter our `DataFrame` in a lot of ways. You've already seen the `.query()` method, and there's more to it.

But there's also the column masking method. Essentially this filters the `DataFrame` by applying a **mask**—a Series of boolean values—to a `DataFrame`, and only returning rows where the Series value is `True`.

The syntax is a little funky, so let's go through it step by step.

First, let's explain the mask. We can create a mask by using a boolean expression about a Series. For example, comparing `.source_ip` to `"10.0.1.5"`:

In [35]:
df.source_ip == "10.0.1.5"

0       True
1       True
2       True
3       True
4       True
       ...  
166    False
167    False
168    False
169    False
170    False
Name: source_ip, Length: 171, dtype: bool

What we get back is a Series containing bools! If we put this expression inside of square braces after our `DataFrame` name, we're telling Pandas to give us rows not blocked by this mask!

In [39]:
# Mask the df and get source_ip and query cols
df[df.source_ip == "10.0.1.5"][["source_ip", "query"]]

,source_ip,query
0,10.0.1.5,ctldl.windowsupdate.com
1,10.0.1.5,ctldl.windowsupdate.com
2,10.0.1.5,arc.msn.com
3,10.0.1.5,img-prod-cms-rt-microsoft-com.akamaized.net
4,10.0.1.5,img-prod-cms-rt-microsoft-com.akamaized.net
...,...,...
143,10.0.1.5,rdfepirv2bl3prdstr02.blob.core.windows.net
144,10.0.1.5,_ldap._tcp.default-first-site-name._sites.dc._...
145,10.0.1.5,login.live.com
146,10.0.1.5,watson.telemetry.microsoft.com


### Working with Series Data

Sometimes, the data in a Series is a little...finicky. Many data types like `str` or `datetime` have methods for working with them using the mask method.

For example, what if we wanted to mask on strings ending with a certain value? Pandas has a [`.str.endswith()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.endswith.html) method that does the job, but we have to use it properly.

> Pandas, why are you like this?

I know, it's annoying. It's because we're _never working with one value at a time_. This is a fundamental shift in our thinking about data. When working with `DataFrame`s, we have to consider what we're doing across rows and columns.

In our count of queries by source, you may have noticed a bunch of `.dmevals.local` domains. These are internal. If we're looking for external DNS queries, we can safely exclude them.

But wait. We can use `.str.endswith()` to find matches, but we want the _inverse_ match! How do?

Prepending a mask expression with `~` negates it.

Let's run that mask and save this "slice" as a new `DataFrame`

In [42]:
# Grab our external-only queries
external_queries = df[~df["query"].str.endswith(".dmevals.local")]

# Display the results
external_queries[["source_ip", "query"]]

,source_ip,query
0,10.0.1.5,ctldl.windowsupdate.com
1,10.0.1.5,ctldl.windowsupdate.com
2,10.0.1.5,arc.msn.com
3,10.0.1.5,img-prod-cms-rt-microsoft-com.akamaized.net
4,10.0.1.5,img-prod-cms-rt-microsoft-com.akamaized.net
...,...,...
163,10.0.1.4,v10.events.data.microsoft.com
164,10.0.1.4,v10.events.data.microsoft.com
166,10.0.1.4,v10.events.data.microsoft.com
167,10.0.1.4,settings-win.data.microsoft.com


So obviously there's still some noise there. We don't really need to see the `microsoft.com` queries.

We have the ability to combine masks with boolean-like operators, although they work differently than normal Python `and` abd`or`. Instead, inside the square braces, we use the bitwse `&` and `|`.

Let's add a negation for ending in `microsoft.com` to our mask.

In [45]:
# Grab our external-only queries, minus Microsoft
external_queries = df[~df["query"].str.endswith(".dmevals.local") & ~df["query"].str.endswith(".microsoft.com")]

# Display the first 20 results
external_queries[["source_ip", "query"]].head(20)

,source_ip,query
0,10.0.1.5,ctldl.windowsupdate.com
1,10.0.1.5,ctldl.windowsupdate.com
2,10.0.1.5,arc.msn.com
3,10.0.1.5,img-prod-cms-rt-microsoft-com.akamaized.net
4,10.0.1.5,img-prod-cms-rt-microsoft-com.akamaized.net
5,10.0.1.5,store-images.s-microsoft.com
7,10.0.1.5,nexusrules.officeapps.live.com
8,10.0.1.5,ecs.office.com
9,10.0.1.5,mrodevicemgr.officeapps.live.com
12,10.0.1.5,ecs.office.com


So there's definitely still noise in there, but it's a lot less than there was! And this is how we begin to parse our data to find exactly what we're looking for.

## Check For Understanding

Try some of these challenges to see if you've mastered grouping, aggregation, and masking!

1. What was the average response time (`.rtt`) for queries to `.microsoft.com` domains?
2. What was the least-common external query?
3. What were the top 5 queries for `10.0.1.4`?

In the next lesson, we're going to dive even deeper into Exploratory Data Analysis with some more quantitative techniques!